In [1]:
import pandas as pd
import spacy

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
nltk.download('omw-1.4')

from snowballstemmer import stemmer

ModuleNotFoundError: No module named 'spacy'

### Lecture des fichiers

In [50]:
df = pd.read_csv(r'C:\Users\tlp\Downloads\fullcorpus20210725_DATE.csv', encoding = 'ISO-8859-1').drop('Unnamed: 0',axis=1)
df.replace({'Ã©': 'é', 'Ã¨': 'è', 'Ãª': 'ê', 'Ã´': 'ô','Ã': 'à','Â': '', 'â': "'", 'à ': 'à'}, regex=True, inplace=True)
print(df.shape)
df.head()

(8026, 9)


,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term
0,"Le Figaro, no. 23520",L'agenda caché de Moscou en Italie,2020-03-30,"Lasserre, Isabelle",LES SYMBOLES comptent parfois plus que les fai...,"Sur les camions militaires qui, en colonnes, ...",Deux semaines pour se ressaisir et s'unir ou p...,Figaro,migrant_migratoire
1,Le Monde,Récit Les trois semaines qui ont chamboulé l...,2020-03-27,"àric Albert, Cécile Boutelet, Jérôme Gauthere...","Londres, Berlin, Rome et Bruxelles corresponda...","- Jeudi 26 mars, les Vingt-Sept devaient se r...","pandémie touche tout le monde, indifféremment ...",Monde,migrant_migratoire
2,Le Figaro (site web),àlargissement de l'Union européenne à l'heur...,2020-03-26,"Tandonnet, Maxime",Fin observateur de la vie politique franà§aise...,"contributeur régulier du FigaroVox, Maxime Tan...",es yeux et de se préparer au changement d'un m...,Figaro,migrant_migratoire
3,Le Monde (site web),Coronavirus : les trois semaines qui ont chamb...,2020-03-26,"Eric Albert, Jérôme Gautheret, Cécile Boutelet...","Jeudi 26 mars, les Vingt-Sept doivent se réuni...",n visioconférence pour la troisième fois en mo...,"e la tenue de ses comptes publics, n'est pas s...",Monde,migrant_migratoire
4,Le Monde (site web),Marc Lazar : « Plus l'Italie fait nation face ...,2020-03-25,Marc Lazar,Tribune. L'Italie est le pays qui enregistre à...,date du 25 mars le plus grand nombre de morts...,t de la School of Government de la Luiss (Libr...,Monde,migrant_migratoire


### Préparation des données

In [51]:
# Testing
df = df.loc[df['Journal'].str.contains('Monde', regex=True)].head(100)
df.shape

(100, 9)

In [52]:
# Import du package
import spacy

# pip install numpy=='1.22.1'
# pip install --upgrade h5py # ça m'a debloqué en redémarrant le kernel après ça

# Installation de la pipline entrainée pour la langue française
!python -m spacy download fr_core_news_sm

     ---------------------------------------- 16.3/16.3 MB 7.0 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
nlp = spacy.load("fr_core_news_sm")

In [53]:
textes = df["Texte"]

In [54]:
%%time

spacy_docs = list(nlp.pipe(textes))

Wall time: 23.2 s


Prétraitement  
1) supprimer tous les mots de moins de 3 caractères,  # debatable  
2) supprimer tous les stop-words, et  
3) lemmatiser les mots restants et,  
4) mettre ces mots en minuscules.

In [55]:
%%time

docs = []
for doc in spacy_docs:
    tokens = []
    for token in doc:
        if len(token.orth_) > 3 and not token.is_stop: # prétraitements 1 et 2
            tokens.append( token.lemma_.lower() )  # TODO: prétraitements 3 et 4
    docs.append( tokens )

Wall time: 152 ms


Afin de conserver un peu l'ordre des mots lors de notre modélisation, nous allons tenir en compte les bigrammes fréquents. Pour cela, nous allons utiliser la bibliothèque Gensim (excellente bibliothèque NLP pour les topics modeling et word embeddings)

Voici le processus retenu :

- Identifier les bigrammes fréquents dans le corpus,  
- Ajouter à la liste des tokens pour les documents dans lesquels ils apparaissent.  

Cela signifie que les bigrammes ne seront pas dans leur position correcte dans le texte, mais comme les topic models sont des modèles de bag-of-words (sac-de-mots en français) qui ignorent la position des mots, cela ne pose pas de problèmes

In [56]:
import re
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)

for index in range(len(docs)):
    for token in bigram[docs[index]]:
        if '_' in token:  # les bigrammes peuvent être reconnus par "_" qui concatène les mots
            docs[index].append(token) # TODO

In [57]:
from gensim.corpora import Dictionary

dictionary = Dictionary()

# Ajouter les documents un par un au dictionnaire
for doc in docs:
    dictionary.add_documents([doc])
    
print('Nombre de mots unique dans les documents initiaux :', len(dictionary))

# dictionary.filter_extremes(no_below=3, no_above=0.25)
# print('Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents :', len(dictionary))

print("Exemple :", dictionary.doc2bow(docs[4]))

Nombre de mots unique dans les documents initiaux : 10065
Exemple : [(3, 2), (24, 1), (33, 1), (61, 1), (95, 1), (99, 1), (115, 2), (118, 1), (124, 2), (130, 3), (131, 4), (132, 1), (138, 1), (150, 1), (167, 1), (187, 2), (190, 1), (192, 1), (193, 6), (194, 4), (200, 1), (215, 1), (216, 1), (220, 2), (221, 1), (224, 12), (226, 1), (228, 1), (230, 1), (235, 1), (242, 1), (267, 1), (271, 1), (293, 1), (301, 1), (304, 1), (311, 1), (313, 1), (318, 1), (319, 1), (324, 4), (333, 2), (335, 2), (341, 1), (353, 1), (355, 1), (357, 4), (358, 1), (366, 8), (376, 6), (377, 1), (378, 1), (398, 1), (411, 1), (433, 2), (435, 2), (450, 3), (460, 1), (461, 2), (464, 1), (495, 1), (496, 1), (500, 1), (514, 1), (515, 2), (518, 4), (520, 1), (527, 1), (531, 3), (542, 1), (549, 1), (577, 1), (595, 2), (607, 2), (609, 1), (612, 1), (620, 1), (622, 1), (631, 1), (644, 1), (663, 1), (682, 3), (700, 2), (709, 1), (719, 1), (721, 2), (738, 1), (739, 1), (744, 1), (748, 1), (751, 1), (754, 1), (763, 1), (774, 2

In [58]:
corpus = [ dictionary.doc2bow(doc) for doc in docs] # TODO

In [59]:
%%time
from gensim.models import LdaModel

model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, chunksize=1000, passes=5, random_state=1) # A REMPLIR

Wall time: 1.55 s


In [60]:
for (topic, words) in model.print_topics():
    print("***********")
    print("* topic", topic+1, "*")
    print("***********")
    print(topic+1, ":", words)
    print()

***********
* topic 1 *
***********
1 : 0.006*"pays" + 0.005*"politique" + 0.005*"grand" + 0.004*"france" + 0.004*"européen" + 0.004*"femme" + 0.004*"parti" + 0.003*"europe" + 0.003*"gouvernement" + 0.003*"international"

***********
* topic 2 *
***********
2 : 0.004*"pays" + 0.004*"film" + 0.003*"européen" + 0.003*"monde" + 0.003*"euro" + 0.003*"aurélien_bellanger" + 0.003*"roman" + 0.003*"aurélien" + 0.003*"bellanger" + 0.003*"aujourd'hui"

***********
* topic 3 *
***********
3 : 0.009*"européen" + 0.008*"europe" + 0.006*"pays" + 0.005*"faire" + 0.005*"falloir" + 0.005*"vouloir" + 0.005*"migrant" + 0.004*"puissance" + 0.004*"bien" + 0.003*"personne"

***********
* topic 4 *
***********
4 : 0.007*"pays" + 0.006*"allemand" + 0.005*"président" + 0.004*"faire" + 0.004*"politique" + 0.004*"allemagne" + 0.003*"pouvoir" + 0.003*"trump" + 0.003*"automobile" + 0.003*"européen"

***********
* topic 5 *
***********
5 : 0.008*"droite" + 0.006*"extrême" + 0.006*"russie" + 0.006*"européen" + 0.005

Visualisation

In [61]:
import pyLDAvis.gensim_models
import warnings

pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)

C:\Users\tlp\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.010758  0.009830       1        1   5.505136
1      0.063561  0.087797       2        1   4.925255
2     -0.018696 -0.028561       3        1   4.956703
3     -0.009935  0.077378       4        1  13.106446
4     -0.037493 -0.022715       5        1   8.624360
5     -0.085934 -0.010240       6        1  27.192404
6      0.155789 -0.088755       7        1   8.836857
7     -0.045474 -0.023781       8        1  12.478812
8     -0.073668 -0.042808       9        1   8.931386
9      0.062608  0.041856      10        1   5.442643, topic_info=          Term        Freq       Total Category  logprob  loglift
194   européen  329.000000  329.000000  Default  30.0000  30.0000
1029     parti  158.000000  158.000000  Default  29.0000  29.0000
9259   abaaoud   29.000000   29.000000  Default  28.0000  28.0000
366       pays  278.000000  278.000000  Default  27.0000  27.0000
1195    droite  139.000000  139.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
808     social    5.564447   58.613159  Topic10  -6.1625   0.5563
1194     droit    5.698764   87.263076  Topic10  -6.1386   0.1822
220     france    5.427698  127.086217  Topic10  -6.1874  -0.2424
61        bien    5.338732  119.084161  Topic10  -6.2039  -0.1939
744     passer    5.146826   70.243306  Topic10  -6.2405   0.2973

[732 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
853       4  0.524965      
853       5  0.080764      
853       6  0.161528      
853       7  0.080764      
853       8  0.080764      
...     ...       ...   ...
559       6  0.400326  être
559       7  0.084279  être
559       8  0.084279  être
559       9  0.063209  être
559      10  0.063209  être

[1936 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [62]:
# Nous en affichons que 4
n_doc = 4
i = 0
for (text, doc) in zip(textes[:n_doc], docs[:n_doc]):
    i += 1
    print("***********")
    print("* doc", i, "  *")
    print("***********")
    print(text)
    print([(topic+1, prob) for (topic, prob) in model[dictionary.doc2bow(doc)] if prob > 0.1])
    print()

***********
* doc 1   *
***********
Londres, Berlin, Rome et Bruxelles correspondants - Jeudi 26 mars, les Vingt-Sept devaient se réunir en visioconférence pour la troisième fois en moins de trois semaines. Au programme des discussions, la crise due au coronavirus, alors que l'Europe entière à  l'exception de la Suède est confinée, que des centaines de malades meurent tous les jours et que l'économie continentale est en voie de paralysie.   Si l'urgence est bien sà»r sanitaire, ils se retrouvent alors que les tabous économiques européens tombent les uns après les autres. En quelques semaines, le pacte de stabilité et de croissance, qui régit la sacro-sainte discipline budgétaire des pays de la zone euro, a été suspendu; l'Allemagne a présenté le plus gros budget de relance de son histoire et renoncé à  l'équilibre budgétaire pourtant inscrit dans le marbre; la Commission européenne a mis des dizaines de milliards sur la table; la Banque centrale européenne (BCE) a annoncé l'injection d